In [ ]:
import json, os
import cv2
import torch
from multiprocessing import Pool, Manager
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.akpd import AKPD
from aquabyte.template_matching import find_matches_and_homography
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from aquabyte.akpd_scorer import generate_confidence_score
from keras.models import load_model
import boto3
import pandas as pd
import numpy as np
import plotly.express as px
import time
from matplotlib import pyplot as plt

from collections import defaultdict
import datetime as dt
import json
import numpy as np
from sklearn.linear_model import LinearRegression
from collections import defaultdict



In [ ]:
s3_access_utils = S3AccessUtils('/root/data')




In [ ]:
# extract weight dataframe
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
pen_id, group_id = 57, '57_t3_akpd'
query = """
    SELECT * FROM
    prod.biomass_computations bc
    WHERE bc.pen_id={0}
    AND (bc.group_id='{1}' OR bc.group_id='{0}')
    AND bc.captured_at between '2019-12-05' and '2019-12-15'
    AND bc.akpd_score > 0.9;
""".format(pen_id, group_id)

wdf = rds_access_utils.extract_from_database(query)
wdf = wdf.sort_values('captured_at')
wdf.index = pd.to_datetime(wdf.captured_at)

# extract lice dataframe
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
pen_id = 57
query = """
    SELECT * FROM
    lati_fish_detections_lice_annotations_reconciled
    WHERE pen_id={0}
    AND captured_at between '2019-12-05' and '2019-12-15'
""".format(pen_id)

ldf = rds_access_utils.extract_from_database(query)
ldf = ldf.sort_values('captured_at')
ldf.index = pd.to_datetime(ldf.captured_at)


In [ ]:
weights = []
for idx, row in ldf.iterrows():
    left_crop_url = row.image_url
    if (wdf.left_crop_url == left_crop_url).sum() > 0:
        weight = wdf[wdf.left_crop_url == left_crop_url].estimated_weight_g.iloc[0]
        weights.append(weight)
    else:
        weights.append(None)
        
ldf['weight'] = weights

In [ ]:
for lice_count in list(range(10)):
    mean_weight = ldf[ldf.adult_female_count_adjusted == lice_count].weight.mean()
    print(mean_weight)

In [ ]:
weights = np.arange(500, 2000, 100)

In [ ]:
for idx in range(len(weights) - 1):
    mask = (ldf.weight > weights[idx]) & (ldf.weight < weights[idx+1])
    mean_lice_count = ldf[mask].adult_female_count_adjusted.mean()
    print(mean_lice_count)
    